In [213]:
import numpy as np
import uproot
from copy import deepcopy
import os
import glob
import plotly.graph_objs as go
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, Output
from IPython.display import display, clear_output
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.optimize import fmin
import matplotlib.pyplot as plt
import pandas as pd
import particle


In [214]:
df = uproot.open("../build/test.root:Hits")["pixelHits"].arrays(library="pd")
geom = uproot.open("../build/test.root:geometry").arrays(library="np")

In [215]:
print(df.iloc[0])
row = df.iloc[0]
# long_df = pd.DataFrame({
#     "x": row["hit_rowID"],
#     "y": row["hit_colID"],
#     "z": row["hit_layerID"],
#     "pdg": row["hit_pdgc"]
# })

long_df = pd.DataFrame({
    "x": [geom['pixel_xpos'][0][int(i)] for i in row["hit_rowID"]],
    "y": [geom['pixel_Ypos'][0][int(i)] for i in row["hit_colID"]],
    "z": [geom['pixel_Zpos'][0][int(i)] for i in row["hit_layerID"]],
    "pdg": row["hit_pdgc"],
    "isPrimary": row["hit_fromPrimaryLepton"]
})

print("Particle species: ", set(long_df["pdg"]))

event_id                                                                 0
hit_rowID                [4810.0, 4811.0, 4811.0, 4812.0, 4813.0, 4823....
hit_colID                [4875.0, 4875.0, 4876.0, 4876.0, 4876.0, 4877....
hit_layerID              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
hit_pdgc                 [-11, -11, -11, -11, -11, -11, -11, -11, -11, ...
hit_trackID              [268075, 268075, 268075, 268075, 268075, 26807...
hit_parentID             [248598, 248598, 248598, 248598, 248598, 24859...
hit_px                   [-4.0102286, -4.0990887, -4.151183, -4.1959634...
hit_py                   [-2.01026, -1.9419761, -1.9428697, -1.9254926,...
hit_pz                   [3.405609, 3.3650458, 3.309684, 3.2708156, 2.9...
hit_energy               [5.655296, 5.670842, 5.6764593, 5.6809316, 5.6...
hit_charge               [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
hit_fromPrimaryPizero    [False, False, False, False, False, False, Fal...
hit_fromFSLPizero        

In [216]:
def pdg_to_particle(pdg):
    if abs(pdg) == 11:
        return "electron"
    if abs(pdg) == 13:
        return "muon"
    if abs(pdg) == 15:
        return "tau"
    return "other"

long_df["particle"] = long_df["pdg"].apply(pdg_to_particle)


In [217]:
def pdg_to_name(pdg):
    return particle.Particle.from_pdgid(pdg).name

long_df["name"] = long_df["pdg"].apply(pdg_to_name)


In [218]:
def isPrim_to_particle(isPrim):
    if isPrim > 0:
        return "Primary"
    else:
        return "Secondary"

long_df["primary"] = long_df["isPrimary"].apply(isPrim_to_particle)

In [219]:
import plotly.express as px

fig = px.scatter_3d(
    long_df,
    x="x",
    y="y",
    z="z",
    color="particle",
    color_discrete_map={
        "electron": "blue",
        "muon": "orange",
        "tau": "purple",
        "other": "gray"
    },
    opacity=0.8,
    hover_name="name"
)

fig.update_traces(marker=dict(size=1))
fig.show()


In [220]:
import plotly.express as px

fig = px.scatter_3d(
    long_df,
    x="x",
    y="y",
    z="z",
    color="primary",
    color_discrete_map={
        "Primary": "orange",
        "Secondary": "gray"
    },
    opacity=0.8,
    hover_name="name"

)

fig.update_traces(marker=dict(size=1))
fig.show()
